In [ ]:
# Local imports
import os
import glob
import re
import sys
import argparse

# Third-party imports
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.cm as mpl_cm
import matplotlib
import cartopy.crs as ccrs
import iris
import iris.coord_categorisation as coord_cat
import iris.plot as iplt
import scipy
import pdb
import datetime
import iris.quickplot as qplt

# Import dictionaries and functions from my module
import dictionaries as dic
import functions as fnc

In [ ]:
# Set up the variables
model = "HadGEM3-GC31-MM"
variable = "psl"
region = "global"
season = "DJFM"
forecast_range = "2-9"
start_year = "1960"
end_year = "2014"
obs_path = dic.obs_path

In [ ]:
# Process the observations
obs_anomaly = fnc.read_obs(variable, region, forecast_range,
                            season, obs_path, start_year, end_year)

In [ ]:
# Load the model cube
anom_mm = fnc.load_model_cube(variable, region, season, forecast_range)